# Классификация изображений

### Основная идея этого решения: взять предобученую на ImageNet сеть Xception и дообучить под нашу задачу. 


In [ ]:
!nvidia-smi

In [1]:
!pip install - -upgrade efficientnet

Requirement already up-to-date: efficientnet in /Users/paulmatus/opt/anaconda3/lib/python3.8/site-packages (1.1.1)


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.io
import tarfile
import csv
import sys
import os

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
import tensorflow.keras.callbacks as C
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import optimizers
import efficientnet.tfkeras as efn
import zipfile

from ImageDataAugmentor.image_data_augmentor import *
import albumentations

from sklearn.model_selection import train_test_split

import PIL
from PIL import ImageOps, ImageFilter
# увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
# графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

print(os.listdir("../input"))
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

**Работаем с Tensorflow v2**

In [ ]:
!pip freeze > requirements.txt

# Основные настройки

# В setup выносим основные настройки: так удобнее их перебирать в дальнейшем.



In [ ]:
EPOCHS = 20  # эпох на обучение
# уменьшаем batch если сеть большая, иначе не влезет в память на GPU  (Для модели B6 делаем 8, иначе в память не влезет )
BATCH_SIZE = 8
LR = 1e-4  # можно взять меньше
VAL_SPLIT = 0.20  # сколько данных выделяем на тест от 15до 30%
CLASS_NUM = 10  # количество классов в нашей задаче
IMG_SIZE = 299  # какого размера подаем изображения в сеть
IMG_CHANNELS = 3  # у RGB 3 канала
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
DATA_PATH = '../input/'
PATH = "../working/car/"  # рабочая директория

In [ ]:
# Устаналиваем конкретное значение random seed для воспроизводимости
# os.makedirs(PATH,exist_ok=False)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
PYTHONHASHSEED = 0

# EDA / Анализ данных

In [ ]:
train_df = pd.read_csv(DATA_PATH+"train.csv")
sample_submission = pd.read_csv(DATA_PATH+"sample-submission.csv")
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.Category.value_counts()
# распределение классов достаточно равномерное - это хорошо

In [ ]:
print('Распаковываем картинки')
# Will unzip the files so that you can see them..
for data_zip in ['train.zip', 'test.zip']:
    with zipfile.ZipFile("../input/"+data_zip, "r") as z:
        z.extractall(PATH)

print(os.listdir(PATH))

In [ ]:
print('Пример картинок (random sample)')
plt.figure(figsize=(12, 8))

random_image = train_df.sample(n=9)
random_image_paths = random_image['Id'].values
random_image_cat = random_image['Category'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(PATH+f'train/{random_image_cat[index]}/{path}')
    plt.subplot(3, 3, index+1)
    plt.imshow(im)
    plt.title('Class: '+str(random_image_cat[index]))
    plt.axis('off')
plt.show()

Посмотрим на примеры картинок и их размеры чтоб понимать как их лучше обработать и сжимать.

In [ ]:
image = PIL.Image.open(PATH+'/train/0/100380.jpg')
imgplot = plt.imshow(image)
plt.show()
image.size

# Подготовка данных

### Аугментация данных

In [ ]:
# Аугментация данных важна, когда мы работаем с небольшим датасетом. Это как раз наш случай.
# Прообуем 2 варианта аугментации
# Вариант №1

# train_datagen = ImageDataGenerator(
# rescale=1. / 255,
#  rotation_range = 6,
# width_shift_range=0.1,
# height_shift_range=0.2,
# validation_split=VAL_SPLIT, # set validation split
# horizontal_flip=False)

#test_datagen = ImageDataGenerator(rescale=1. / 255)

# Рекомендация Подключите более продвинутые библиотеки аугментации изображений (например: albumentations или imgaug, для них есть специальные "обертки" под Keras, например: https://github.com/mjkvaak/ImageDataAugmentor)

In [ ]:
# Вариант аугментации №2, более продвинутый ImageDataAugmentor на базе albumentations
# его и будем использовать:

In [ ]:
pip install git+https: // github.com/mjkvaak/ImageDataAugmentor

In [ ]:
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomContrast, RandomBrightness, Flip, OneOf, Compose
)

In [ ]:
from ImageDataAugmentor.image_data_augmentor import *
import albumentations

AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5),
    albumentations.Rotate(limit=45, interpolation=1, border_mode=4,
                          value=None, mask_value=None, always_apply=False, p=0.5),
    albumentations.OneOf([
        albumentations.CenterCrop(height=250, width=200),
        albumentations.CenterCrop(height=200, width=250),
    ], p=0.5),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(
            brightness_limit=0.1, contrast_limit=0.1)
    ], p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.HueSaturationValue(p=0.5),
    albumentations.RGBShift(p=0.5),
    albumentations.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
    albumentations.Resize(IMG_SIZE, IMG_SIZE)
])

train_datagen = ImageDataAugmentor(
    rescale=1./255,
    augment=AUGMENTATIONS,
    validation_split=VAL_SPLIT,
)

test_datagen = ImageDataAugmentor(rescale=1./255)

### Генерация данных

In [ ]:
# Завернем наши данные в генератор:

train_generator = train_datagen.flow_from_directory(
    PATH+'train/',      # директория где расположены папки с картинками
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='training')  # set as training data

test_generator = train_datagen.flow_from_directory(
    PATH+'train/',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True, seed=RANDOM_SEED,
    subset='validation')  # set as validation data

test_sub_generator = test_datagen.flow_from_dataframe(
    dataframe=sample_submission,
    directory=PATH+'test_upload/',
    x_col="Id",
    y_col=None,
    shuffle=False,
    class_mode=None,
    seed=RANDOM_SEED,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,)

# Построение модели

In [ ]:
# Можно построить на базе предобученной сети Xception:
# base_model = Xception(weights='imagenet', include_top=False, input_shape = input_shape)

In [ ]:
# Но мы будем использовать другую сеть, которая показыает лучше результаты "EfficientNetB6"
import efficientnet.tfkeras as efn
input_shape = (IMG_SIZE, IMG_SIZE, IMG_CHANNELS)
base_model = efn.EfficientNetB6(
    weights='imagenet',  # Подгружаем веса imagenet
    include_top=False,
    input_shape=input_shape
)  # Выходной слой (голову) будем менять т.к. у нас други классы input_shape = input_shape)

In [ ]:
base_model.summary()

In [ ]:
# Устанавливаем новую "голову" (head):
model = M.Sequential()
model.add(base_model)
# объединяем все признаки в единый вектор
model.add(L.GlobalAveragePooling2D(),)
# Экспериментируем с архитектурой - добавляем  один (от 1 до 4 плотных) полносвязный слой, dropout и batch-нормализацию
# elu ,leaky_ReLU, sigmoid, tanh - как альтернативные варианты
model.add(L.Dense(256, activation='relu'))
model.add(L.Dropout(0.20))
model.add(L.BatchNormalization())
model.add(L.Dense(128, activation='relu'))
model.add(L.Dropout(0.30))
model.add(L.BatchNormalization())
#model.add(L.Dense(64, activation='relu'))
#model.add(L.Dropout(0.35))
#model.add(L.BatchNormalization())
#model.add(L.Dense(64, activation='relu'))
#model.add(L.Dropout(0.45))
#model.add(L.BatchNormalization())
model.add(L.Dense(CLASS_NUM, activation='softmax'))  # "sigmoid" как вариант
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(lr=LR), metrics=["accuracy"])

In [ ]:
model.summary()
# Рекомендация: Попробуйте добавить Batch Normalization

## Обучение модели

Добавим ModelCheckpoint чтоб сохранять прогресс обучения модели и можно было потом подгрузить и дообучить модель.

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5', monitor=[
                             'val_accuracy'], verbose=1, mode='max')
callbacks_list = [checkpoint]

# Рекомендация 1. Добавьте другие функции из https://keras.io/callbacks/
# Рекомендация 2. Используйте разные техники управления Learning Rate
# https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng)
# http://teleported.in/posts/cyclic-learning-rate/ (eng)

In [ ]:
# Добавим технику EarlyStopping
from keras.callbacks import EarlyStopping
earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
callbacks_list = [checkpoint, earlystop]

Обучаем:

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=EPOCHS,
    callbacks=callbacks_list
)
# Рекомендация: попробуйте применить transfer learning с fine-tuning - изучил, но не успел реализовать.

In [ ]:
# сохраним итоговую сеть и подгрузим лучшую итерацию в обучении (best_model)
model.save('../working/model_last.hdf5')
model.load_weights('best_model.hdf5')

In [ ]:
scores = model.evaluate_generator(
    test_generator, steps=len(test_generator), verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

В Итоге точность нашей модели составила 96.55%. 
Учитывая что классов 10 - это Очень хороший результат!     
Посмотрим графики обучения:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Предсказание на тестовых данных

In [ ]:
test_sub_generator.samples

In [ ]:
test_sub_generator.reset()
predictions = model.predict_generator(
    test_sub_generator, steps=len(test_sub_generator), verbose=1)
predictions = np.argmax(predictions, axis=-1)  # multiple categories
label_map = (train_generator.class_indices)
label_map = dict((v, k) for k, v in label_map.items())  # flip k,v
predictions = [label_map[k] for k in predictions]

In [ ]:
filenames_with_dir = test_sub_generator.filenames
submission = pd.DataFrame(
    {'Id': filenames_with_dir, 'Category': predictions}, columns=['Id', 'Category'])
submission['Id'] = submission['Id'].replace('test_upload/', '')
submission.to_csv('submission.csv10', index=False)
print('Save submit')

# Рекомендация: попробуйте добавить Test Time Augmentation (TTA)  - не успел.
# https://towardsdatascience.com/test-time-augmentation-tta-and-how-to-perform-it-with-keras-4ac19b67fb4d

In [ ]:
submission.to_csv('/kaggle/working/submission14.csv', index=False)

In [ ]:
submission.head()

In [ ]:
# Clean PATH
import shutil
shutil.rmtree(PATH)

# Что можно было сделать, чтобы улучшить результат:

* Примените transfer learning с fine-tuning
* Подберите другие переменные (размер картинки, батч и т.д.)
* Добавьте TTA (Test Time Augmentation)
* Дополнительно*: Используйте разные техники управления Learning Rate (https://towardsdatascience.com/finding-good-learning-rate-and-the-one-cycle-policy-7159fe1db5d6 (eng) http://teleported.in/posts/cyclic-learning-rate/ (eng))
* Дополнительно*: Добавьте более продвинутые библиотеки аугментации изображений (например, Albumentations )



# выводы:
Построили вполне хорошую модель, от победителя отстали менее 1%, при этом система не такая нагруженная и быстрее в 4 раза.
Занял 26 место из 97 на Kaggle.com